In [27]:
import time

import numpy as np

def rotate_householder0(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    x is a unit vector, so shape is (n_dims,)
    """
    assert S.ndim == 2
    assert x.ndim == 1
    assert y.ndim == 1
    assert S.shape[1] == x.shape[0] == y.shape[0]
    # 1. First reflection: x -> -x
    u1 = x / np.linalg.norm(x)
    
    # 2. Second reflection: -x -> y
    v2 = y - (-x)
    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    S_temp = S - 2 * np.outer(S @ u1, u1)
    S_rotated = S_temp - 2 * np.outer(S_temp @ u2, u2)
    
    return S_rotated


def rotate_householder1(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    x is a unit vector, so shape is (n_dims,)
    """
    assert S.ndim == 2
    assert x.ndim == 1
    assert y.ndim == 1
    assert S.shape[1] == x.shape[0] == y.shape[0]
    # 1. First reflection: x -> -x
    u1 = x / np.linalg.norm(x)
    
    # 2. Second reflection: -x -> y
    v2 = y - (-x)
    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    S_temp = S - 2 * np.outer(np.dot(S, u1), u1)
    S_rotated = S_temp - 2 * np.outer(np.dot(S_temp, u2), u2)
    
    return S_rotated


def rotate_householder2(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    x is a unit vector, so shape is (n_dims,)
    """
    assert S.ndim == 2
    assert x.ndim == 1
    assert y.ndim == 1
    assert S.shape[1] == x.shape[0] == y.shape[0]
    # 1. First reflection: x -> -x
    u1 = x / np.linalg.norm(x)
    
    # 2. Second reflection: -x -> y
    v2 = y - (-x)
    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    S_temp = S - np.outer(2 * np.dot(S, u1), u1)
    S_rotated = S_temp - np.outer(2 * np.dot(S_temp, u2), u2)
    
    return S_rotated


def rotate_householder3(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    x is a unit vector, so shape is (n_dims,)
    """
    assert S.ndim == 2
    assert x.ndim == 1
    assert y.ndim == 1
    assert S.shape[1] == x.shape[0] == y.shape[0]
    # 1. First reflection: x -> -x
    u1 = x
    
    # 2. Second reflection: -x -> y
    v2 = y.copy()
    v2[0] += x[0]

    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    S_dot = np.empty((S.shape[0],), dtype=S.dtype)
    S_outer = np.empty_like(S)

    np.dot(S, u1, out=S_dot)
    np.multiply(2, S_dot, out=S_dot)
    np.outer(S_dot, u1, out=S_outer)
    S -= S_outer
    np.dot(S, u2, out=S_dot)
    np.multiply(2, S_dot, out=S_dot)
    np.outer(S_dot, u2, out=S_outer)
    return S - S_outer

def rotate_householder4(x, y, S):
    """
    Applies the shortest-path rotation mapping x -> y to set S.
    S is an array of shape (n_samples, n_dims).
    x is a unit vector, so shape is (n_dims,)
    """
    assert S.ndim == 2
    assert x.ndim == 1
    assert y.ndim == 1
    assert S.shape[1] == x.shape[0] == y.shape[0]
    # 1. First reflection: x -> -x
    u1 = x
    
    # 2. Second reflection: -x -> y
    v2 = y.copy()
    v2[0] += x[0]

    u2 = v2 / np.linalg.norm(v2)
    
    # Vectorized application: H(s) = s - 2(u·s)u
    # We apply H1 then H2
    # S_dot = np.empty((S.shape[0],), dtype=S.dtype)
    S_outer = np.empty_like(S)

    S_dot = S[:, 0].copy()  # We can use the structure of x to make this faster (skipping the dot product)
    np.multiply(2, S_dot, out=S_dot)
    np.outer(S_dot, u1, out=S_outer)
    S -= S_outer
    np.dot(S, u2, out=S_dot)
    np.multiply(2, S_dot, out=S_dot)
    np.outer(S_dot, u2, out=S_outer)
    return S - S_outer

# --- Validation ---
n = 50_000
size = 2_000
iterations = 1
x = np.zeros((n,))
x[0] = 1.0
y = np.random.randn(n)
y /= np.linalg.norm(y)

# Set S: a point at 30 degrees (pi/6)
S = np.random.randn(size, n)
S /= np.linalg.norm(S, axis=-1, keepdims=True)

Ss = []
methods = {
    "rotate_householder0          ": (rotate_householder0, False),
    "rotate_householder1          ": (rotate_householder1, False), 
    "rotate_householder2          ": (rotate_householder2, False), 
    "rotate_householder3          ": (rotate_householder3, True),
    "rotate_householder3 (no copy)": (rotate_householder3, False),
    "rotate_householder4          ": (rotate_householder4, True),
    "rotate_householder4 (no copy)": (rotate_householder4, False)
}

# Measure the runtime of each method
for method_name, (method, make_copy) in methods.items():
    _S = S.copy()
    start = time.time()
    for i in range(iterations):
        if make_copy:
            S_method = method(x, y, S.copy())
        elif i == iterations - 1:
            S_method = method(x, y, _S)
        else:
            S_method = method(x, y, S)
    end = time.time()
    print(f"{method_name} time: {(end - start) / iterations:.6f}")
    Ss.append(S_method)
    del S_method, _S


S_hh = Ss[0]
S_hh_improved = Ss[1:]

# Check that all the improved methods are close to the original
for S_improved in S_hh_improved:
    print(np.allclose(S_hh, S_improved))


rotate_householder0           time: 0.720068
rotate_householder1           time: 0.733792
rotate_householder2           time: 0.574054
rotate_householder3           time: 0.610965
rotate_householder3 (no copy) time: 0.487386
rotate_householder4           time: 1.077663
rotate_householder4 (no copy) time: 0.498099
True
True
True
True
True
True


In [ ]:
def rotate_set_qr(x, y, S):
    """
    Maps x -> y using QR Decomposition. 
    Complexity: O(n^3) to find Q, O(n^2) per vector in S.
    """
    n = len(x)
    # Create a basis starting with y
    M = np.eye(n)
    M[:, 0] = y
    
    # QR gives an orthogonal matrix Q where Q[:, 0] is proportional to y
    Q, _ = np.linalg.qr(M)
    
    # QR doesn't guarantee the sign of the first column. Fix it:
    if not np.allclose(Q[:, 0], y):
        Q[:, 0] = -Q[:, 0] # This might flip det to -1

    # Force det(Q) == 1 to ensure pure rotation (not reflection)
    if np.linalg.det(Q) < 0:
        Q[:, -1] = -Q[:, -1]
        
    # Rotate S: S_rotated = S * Q^T
    return S @ Q.T

n = 1_000
size = 100
S = np.random.randn(size, n)
S /= np.linalg.norm(S, axis=-1, keepdims=True)

x = np.zeros((n,))
x[0] = 1.0
y = np.random.randn(n)

S_qr_rotated = rotate_set_qr(x, y, S)
S_hh_rotated = rotate_householder4(x, y, S.copy())



In [75]:
import numpy as np

# --- 1. The Transformation Functions ---

def get_householder_rotation(x, y):
    """Returns a function that applies the Double Householder rotation."""
    # Handle alignment edge cases
    if np.allclose(x, y): return lambda s: s
    if np.allclose(x, -y):
        # To map x -> -x via planar rotation, we need an auxiliary vector.
        # We'll use the second basis vector to define the rotation plane.
        u1 = x
        u2 = np.zeros_like(x)
        # If x is already the second basis vector, use the first.
        u2[1] = 1.0
        
    else:
        # Standard Shortest-Path Geodesic
        u1 = x / np.linalg.norm(x)
        v2 = y + x 
        u2 = v2 / np.linalg.norm(v2)

    def rotate(S):
        # Handle both single vectors and matrices
        is_single = S.ndim == 1
        S_arr = np.atleast_2d(S)
        
        # H1: Reflect across u1
        S_temp = S_arr - 2 * np.outer(S_arr @ u1, u1)
        # H2: Reflect across u2
        S_rotated = S_temp - 2 * np.outer(S_temp @ u2, u2)
        
        return S_rotated.squeeze() if is_single else S_rotated
    
    return rotate

def get_qr_rotation(x, y):
    """Returns a function that applies the QR-based rotation."""
    n = len(x)
    M = np.eye(n)
    M[:, 0] = y
    Q, _ = np.linalg.qr(M)
    
    # Fix determinant to ensure rotation (+1) not reflection (-1)
    if np.linalg.det(Q) < 0:
        Q[:, -1] *= -1
        
    # Check if we accidentally flipped the alignment (Qx should be y)
    # Since x is [1,0...], Qx is the first column.
    # If QR made the first col -y, flip it back.
    if np.dot(Q[:,0], y) < 0:
         Q[:, 0] = -Q[:, 0]
         # Flipping one col flips det sign, so flip last col again to keep det=1
         Q[:, -1] = -Q[:, -1]

    def rotate(S):
        # Apply matrix Q (transpose because we treat S as row vectors)
        # However, we want R such that R * [1,0,0] = y. 
        # In our construction, Q's first col is y.
        # So Q is the rotation matrix R.
        return np.dot(S, Q.T) # S @ Q.T is equivalent to (Q @ S.T).T
    
    return rotate

# --- 2. The Twist Measurement Function ---

def measure_twist(rotation_func, x, y):
    """
    Measures how much a rotation 'twists' the space perpendicular to the rotation plane.
    Returns the angle of distortion in degrees.
    """
    n = len(x)
    size = 100
    # --- 1. Generate the Probe Vectors ---
    random_vectors = np.random.randn(size, n).astype(x.dtype)
    
    # --- 2. Construct a proper basis for the x-y plane ---
    e1 = x / np.linalg.norm(x)
    
    # Define the rotation plane basis
    v2 = y - np.dot(y, e1) * e1
    if np.linalg.norm(v2) < 1e-10:
        # Case y == x or y == -x: use an arbitrary second vector for the plane
        e2 = np.zeros_like(x)
        e2[0 if np.abs(e1[0]) < 0.9 else 1] = 1.0
        e2 = e2 - np.dot(e2, e1) * e1
        e2 /= np.linalg.norm(e2)
    else:
        e2 = v2 / np.linalg.norm(v2)

    # Project random vectors onto the ORTHOGONAL space of the plane (e1, e2)
    proj = np.outer(random_vectors @ e1, e1) + np.outer(random_vectors @ e2, e2)
    z = random_vectors - proj
    z /= np.linalg.norm(z, axis=1, keepdims=True)
    
    # 2. Apply the rotation to the probe vectors
    z_rotated = rotation_func(z)
    
    # 3. Measure the angle between the original z and rotated z
    # Ideally, a pure x->y rotation should NOT touch z at all (angle = 0).
    # We only need the dot product between z_i and z_rotated_i for each i.
    dot_prods = np.clip(np.sum(z * z_rotated, axis=1), -1.0, 1.0)

    angle_deg = np.degrees(np.arccos(dot_prods))
    
    return angle_deg.mean(), angle_deg.std()

# --- 3. The Execution ---

# Setup
n = 30
x = np.zeros(n, dtype=np.float64); x[0] = 1.0 # Standard basis vector
y = -x # np.random.randn(n); y /= np.linalg.norm(y) # Random target
y = y.astype(np.float64)

# Define the rotators
rot_hh = get_householder_rotation(x, y)
rot_qr = get_qr_rotation(x, y)

# --- Test 1: Verify they both actually map x to y ---
print(f"Target y:     {y[:3]}...")
print(f"HH(x):        {rot_hh(x)[:3]}...")
print(f"QR(x):        {rot_qr(x)[:3]}...")
print("-" * 30)

# --- Test 2: Measure the Twist ---
twist_hh, twist_hh_std = measure_twist(rot_hh, x, y)
twist_qr, twist_qr_std = measure_twist(rot_qr, x, y)

print(f"Twist introduced by Householder: {twist_hh:8.4e} degrees (std: {twist_hh_std:6.4e})")
print(f"Twist introduced by QR Method:   {twist_qr:8.4f} degrees (std: {twist_qr_std:6.4f})")

assert np.isclose(twist_hh, 0, atol=1e-6), "Householder should have 0 twist!"
assert not np.isclose(twist_qr, 0), "QR usually introduces significant twist!"

print("\nConclusion: QR twists the universe; Householder flies straight.")

Target y:     [-1. -0. -0.]...
HH(x):        [-1.  0.  0.]...
QR(x):        [-1.  0.  0.]...
------------------------------
Twist introduced by Householder: 4.8767e-07 degrees (std: 5.9992e-07)
Twist introduced by QR Method:    16.4971 degrees (std: 11.1194)

Conclusion: QR twists the universe; Householder flies straight.
